In [ ]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts

In [ ]:
# Kristina's code starts here
storm_data = pd.read_csv('Resources/clean_storm_event_data.csv')
storm_data = storm_data.drop(columns = 'Unnamed: 0')
storm_data

In [ ]:
# Kristina's code ends here

In [ ]:
# Jeff's code starts here

In [ ]:
# Jeff's code ends here

In [ ]:
# Faduma's code starts here

In [ ]:
# Faduma's code ends here

In [ ]:
# Emily's code starts here

In [ ]:
# Emily's code ends here